# 5.2 Defining interventions

In this tutorial, you will learn how to add interventions to Starsim.

Here, we will define a vaccine intervention that is applied at a single point in time (`timestep`) that increases the immunity of a random subset of agents (`prob`) by a certain amount (`imm_boost`).

In [ ]:
"""
An example of an SIS model with a vaccine intervention
"""

import numpy as np # Math
import sciris as sc # Utilities
import pylab as pl # Plotting
import starsim as ss # ABM

class Vaccine(ss.Intervention): # Create a new, generic vaccine intervention
    
    def __init__(self, timestep=10, prob=0.5, imm_boost=2.0):
        super().__init__() # Initialize the intervention
        self.timestep = timestep # Store the parameters
        self.prob = prob
        self.imm_boost = imm_boost
    
    def apply(self, sim): # Apply the vaccine
        if sim.ti == self.timestep: # Only apply on the matching timestep
            eligible_ids = sim.people.uid # Assume everyone is eligible
            n_eligible = len(eligible_ids) # Number of people who are eligible
            to_vaccinate = self.prob > np.random.rand(n_eligible) # Define which of the n_eligible people get vaccinated by comparing np.random.rand() to self.p
            vaccine_ids = eligible_ids[to_vaccinate] # Pull out the IDs for the people receiving the vaccine
            immunity = sim.diseases.sis.immunity # Shorten the name of the immunity
            immunity[vaccine_ids] += self.imm_boost # Modify the immunity by adding "boost" to the current immunity


def make_run_sim(timestep=10, prob=0.5, imm_boost=2.0): # Create and run the simulation

    vaccine = Vaccine(timestep=timestep, prob=prob, imm_boost=imm_boost) # Create the specific vaccine object
    pars = dict(start=0, end=50, dt=1.0, diseases='sis', networks='random') # Define the parameters of the simulation
    
    # Define "baseline" and "intervention" sims without and with the vaccine
    baseline_sim = ss.Sim(pars)
    vaccine_sim = ss.Sim(pars, interventions=vaccine)
    
    # Run the two simulations
    baseline_sim.run()
    vaccine_sim.run()

    # Pull out results to plot
    time = baseline_sim.yearvec
    baseline_results = baseline_sim.results.sis
    vaccine_results = vaccine_sim.results.sis
    
    n_infected_baseline = baseline_results.n_infected
    n_infected_vaccine = vaccine_results.n_infected
    new_infections_baseline = baseline_results.new_infections
    new_infections_vaccine = vaccine_results.new_infections
    
    # Create the figure
    pl.figure()
    
    # Plot number of people infected
    pl.subplot(2,1,1)
    pl.title('Number of people infected')
    pl.plot(time, n_infected_baseline, 'o-', label='Baseline') # Plot baseline
    pl.plot(time, n_infected_vaccine, 'o-', label='Vaccine') # Plot vaccine
    pl.axvline(time[vaccine.timestep])
    pl.legend()
    
    # Plot number of new infections
    pl.subplot(2,1,2)
    pl.title('New infections')
    pl.plot(time, new_infections_baseline, 'o-', label='Baseline') # Plot baseline
    pl.plot(time, new_infections_vaccine, 'o-', label='Vaccine') # Plot vaccine
    pl.axvline(time[vaccine.timestep])
    pl.legend()

    sc.figlayout()
    pl.show()
    
# Make, run, and plot the simulation
make_run_sim()

## Problem 1

Confirm that with `prob=1.0` and a large boost, that all transmission stops.

## Solution 1

In [ ]:
make_run_sim(prob=1.0, imm_boost=100)

## Problem 2

How do changes in `prob` and `imm_boost` affect the results?

## Solution 2

(Note: these are examples, other parameter values should also be checked.)

In [ ]:
make_run_sim(prob=0.5, imm_boost=2.0)
make_run_sim(prob=1.0, imm_boost=1.0)